In [26]:
#!pip3 install neo4j-driver
#!pip3 install py2neo
!pip3 install neo4j-driver===1.6.2

In [53]:
from neo4j import *
from py2neo import *
from pymongo import MongoClient

In [54]:
# http://localhost:7474
# default password: neo4j
# updated password: neo4j2

# https://neo4j.com/docs/cypher-refcard/3.2/
# https://neo4j.com/docs/cypher-manual/3.5/

In [55]:
db = Database("http://localhost:7474/db/data/", user="neo4j", password="neo4j2")

In [56]:
from py2neo import Graph, Path
graph = Graph(password="neo4j2")

In [57]:
client = MongoClient('localhost', 27017)

In [58]:
db = client.jazz_catalog

In [59]:
db.artists.count()

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


1504

In [60]:
# use MERGE for unique

for artist in db.artists.find():
    graph.run("MERGE (artist:Artist {name:{name}, id:{id}}) RETURN artist", name=artist['name'], id=artist['id'])

In [65]:
c = graph.run("MATCH (n:Artist) WHERE n.name = 'Miles Davis' RETURN n.id")

In [66]:
for i in c:
    print(i)

<Record n.id='561d854a-6a28-4aa7-8c99-323e6ce46c2a'>


In [63]:
# MERGE (instrument:Instrument {name: 'trumpet'}) RETURN instrument.name
# MATCH (a:Artist),(b:Instrument) 
# WHERE a.name = 'Miles Davis' AND b.name = 'trumpet' MERGE (a)-[r:PLAYS { name: a.name + '->' + b.name }]->(b) RETURN type(r), r.name

In [64]:
# set instruments for artists
# create node for instruments and add 'plays' relationship

run_merge = "MERGE (instrument:Instrument {name: {name}}) RETURN instrument.name"
run_match = "MATCH (a:Artist), (b:Instrument) WHERE a.id = {aid} AND b.name = {name} MERGE (a)-[r:PLAYS {name: a.name + '->' + b.name}]->(b)"

for r in db.releases.find():
    if 'medium-list' in r:
        media = r['medium-list']
        for m in media:
            if 'track-list' in m:
                tracks = m['track-list']
                for each in tracks:
                    if 'recording' in each.keys():
                        if 'artist-relation-list' in each['recording'].keys():
                            artist_list = each['recording']['artist-relation-list']
                            for a in artist_list:
                                artist_id = a['artist']['id']
                                if 'attribute-list' in a.keys():
                                    instruments = a['attribute-list']
                                    for inst in instruments:
                                        graph.run(run_merge, name=inst)
                                        graph.run(run_match, aid=artist_id, name=inst)
                                